In [1]:
import sqlalchemy
import pandas as pd
import os
from collections import OrderedDict
from sqlalchemy import create_engine, inspect
import pymysql
pymysql.install_as_MySQLdb()

# Input Files
Source: https://data.world/uscensusbureau/income-poverty-health-ins

Table 3 Poverty Status of People, by Age, Race, and Hispanic Origin: 1959 to 2015
SOURCE: U.S. Bureau of the Census, Current Population Survey, Annual Social and Economic Supplements.

Table 3 People in Poverty by Selected Characteristics: 2014 and 2015


In [2]:
TBL3_POVERTY_BY_RACE = os.path.join('data', 'raw', 'Table_3_Poverty Status of People_Age-Race-Hispanic Origin-hstpov3.xls')
TBL3_POVERTY_BY_SELECTED_CHAR = os.path.join('data', 'raw', 'Table_3_People_Poverty_Selected_Characteristics_2014-2015 .xls')

# Inspect Sheets in Ecel Files

In [3]:
xl_pbr = pd.ExcelFile(TBL3_POVERTY_BY_RACE)
xl_pbsc = pd.ExcelFile(TBL3_POVERTY_BY_SELECTED_CHAR)

In [4]:
xl_pbr.sheet_names

['all races',
 'WHITE, NOT HISPANIC',
 'ASIAN AND PACIFIC ISLANDER',
 'ASIAN ALONE',
 'ASIAN ALONE OR IN COMBINATION',
 'BLACK',
 'BLACK ALONE ',
 'BLACK ALONE OR IN COMBO',
 'WHITE ALONE',
 'WHITE',
 'WHITE ALONE, NOT HISPANIC',
 'HISPANIC, ANY RACE']

In [5]:
xl_pbsc.sheet_names

['All People',
 'Education Attainment',
 'Work Experience',
 'Disability Status',
 'Residence',
 'Age',
 'Sex',
 'Family Status',
 'Race and Hispanic Origin',
 'Naitvity',
 'Region']

# Load Excel Files into DataFrames
This step loads the Excel files into memory as Pandas Data Frames. Here we pass sheet_name parameter as None, which enables load of all worksheets into a dictionary of data frames.

In [6]:
pbr_dfs = pd.read_excel(TBL3_POVERTY_BY_RACE, sheet_name=None)
pbsc_dfs = pd.read_excel(TBL3_POVERTY_BY_SELECTED_CHAR, sheet_name=None)

# Review columns available in all DataFrames
This is just an exploratory step where we try to understand the contents of the Excel workbooks.

In [7]:
print("TBL3_POVERTY_BY_RACE:")
for k, df in pbr_dfs.items():
    print(f"    {k} - {df.columns}\n")
    
print("TBL3_POVERTY_BY_SELECTED_CHARS:")
for k, df in pbsc_dfs.items():
    print(f"    {k} - {df.columns}\n")

TBL3_POVERTY_BY_RACE:
    all races - Index(['ALL RACES- Year and Characteristic',
       'Under 18 years - All People - Total ',
       'Under 18 years - All People - Below Poverty - Number',
       'Under 18 years - All People - Below Poverty - Percent',
       'Under 18 years - Related Children in Families - Total',
       'Under 18 years - Related Children in Families - Below Poverty - Number',
       'Under 18 years - Related Children in Families - Below Poverty - Percent',
       '18 to 64 years - Total', '18 to 64 years - Below Poverty - Number',
       '18 to 64 years - Below Poverty - Percent', '65 years and over - Total',
       '65 years and over - Below Poverty - Number',
       '65 years and over - Below Poverty - Percent'],
      dtype='object')

    WHITE, NOT HISPANIC - Index(['WHITE, NOT HISPANIC- Year and Characteristic',
       'Under 18 years - All People - Total ',
       'Under 18 years - All People - Below Poverty - Number',
       'Under 18 years - All People - 

# Begin ETL Process
In above exploratory step we observed that there are notable structural differences between workbooks and within each workbook across sheets. Our first step in problem solving is to bring the sheets into a common structure and combine the data. Doing this will enable us to extract data to populate various dimension tables.

The ETL process is executed in these sequence of tasks:

1. Transform the structure of Poverty By Race File and combine worksheets
2. Transform the structure of Poverty By Selected Characterestics File and combine worksheets
3. Combine output of above steps to create a consolidated data frame
    . This step creates a data frame which has Total, Below Poverty, Year, Race, Eduction and other selected characterestics.
4. Prepare data for dimension by doing below for each dimension:
    . Access the series associated with the dimension and filter unique values in that series.
    . Create individual dimensions specific data frames with ID, CODE and DESCRIPTION columns
5. Create population data frame by replacing dimension values with corresponding dimention IDs
6. Connect to DB and explore table metadata to understand table names and column names for population table
7. Insert all the dimension data into associated tables
8. Rename columns in population data frame to match DB column names
9. Insert population data into DB

# Process Poverty By Race
In this step we do the following:

    .Rename columns into more meaningful/intuitive names
    .Pivot age data into rows
    .Add Race as a column and use sheetname as the value
    .Concatenate all worksheet specific dfs into one

In [8]:
all_pbr_df = pd.DataFrame({"Year":[], "AgeBand":[], "Race":[], "Total":[], "BelowPoverty":[]})

for key, df in pbr_dfs.items():
    clean_column_names = {df.columns[0]:"Year", 
                          df.columns[1]: "Under 18 - Total", 
                          df.columns[2]: "Under 18 - Below Poverty", 
                          df.columns[3]: "Under 18 - Below Poverty %", 
                          df.columns[4]: "Under 18 - Children Total", 
                          df.columns[5]: "Under 18 - Children Below Poverty", 
                          df.columns[6]: "Under 18 - Children Below Poverty %", 
                          df.columns[7]: "18 to 64 - Total", 
                          df.columns[8]: "18 to 64 - Below Poverty", 
                          df.columns[9]: "18 to 64 - Below Poverty %", 
                          df.columns[10]: "65 and Over - Total", 
                          df.columns[11]: "65 and Over - Below Poverty", 
                          df.columns[12]: "65 and Over - Below Poverty %"}
    clean_df = df.rename(columns=clean_column_names)[["Year",
                    "Under 18 - Total", "Under 18 - Below Poverty",
                    "18 to 64 - Total", "18 to 64 - Below Poverty",
                    "65 and Over - Total", "65 and Over - Below Poverty",
                   ]]
    u18_df = clean_df[["Year", "Under 18 - Total", "Under 18 - Below Poverty"]]
    u18_normalized_columns = {"Under 18 - Total": "Total", "Under 18 - Below Poverty":"BelowPoverty"}
    u18_df = u18_df.rename(columns=u18_normalized_columns)
    u18_df["Race"] = key
    u18_df["AgeBand"] = "Under age 18"
    all_pbr_df = pd.concat([all_pbr_df, u18_df], sort=False)
    
    a1864_df = clean_df[["Year", "18 to 64 - Total", "18 to 64 - Below Poverty"]]
    a1864_normalized_columns = {"18 to 64 - Total": "Total", "18 to 64 - Below Poverty":"BelowPoverty"}
    a1864_df = a1864_df.rename(columns=a1864_normalized_columns)
    a1864_df["Race"] = key
    a1864_df["AgeBand"] = "Aged 18 to 64"
    all_pbr_df = pd.concat([all_pbr_df, a1864_df], sort=False)
    
    a65ao_df = clean_df[["Year", "65 and Over - Total", "65 and Over - Below Poverty"]]
    a65ao_normalized_columns = {"65 and Over - Total": "Total", "65 and Over - Below Poverty":"BelowPoverty"}
    a65ao_df = a65ao_df.rename(columns=a65ao_normalized_columns)
    a65ao_df["Race"] = key
    a65ao_df["AgeBand"] = "Aged 65 and older"
    all_pbr_df = pd.concat([all_pbr_df, a65ao_df], sort=False)
    
print(all_pbr_df.count())
all_pbr_df.head()

Year            933
AgeBand         933
Race            933
Total           933
BelowPoverty    933
dtype: int64


,Year,AgeBand,Race,Total,BelowPoverty
0,1959,Under age 18,all races,64315,17552
1,1960,Under age 18,all races,65601,17634
2,1961,Under age 18,all races,66121,16909
3,1962,Under age 18,all races,67722,16963
4,1963,Under age 18,all races,69181,16005


# Process Poverty By Selected Characterestics
In this step we do the following:

    .Rename columns into more meaningful/intuitive names
    .Pivot year data into rows
    .Prepare a data frame with all measures and dimensions
    .Concatenate all worksheet specific dfs into one

In [9]:
pbsc_dfs.keys()

odict_keys(['All People', 'Education Attainment', 'Work Experience', 'Disability Status', 'Residence', 'Age', 'Sex', 'Family Status', 'Race and Hispanic Origin', 'Naitvity', 'Region'])

In [10]:
all_pbsc_df = pd.DataFrame({"Year":[], "Race":[], 
                            "Education":[], "WorkExperience":[], "DisabilityStatus":[], 
                            "Residence":[], "AgeBand":[], "Nativity":[],
                            "Gender":[], "FamilyStatus":[], "Region":[],
                            "Total":[], "BelowPoverty":[]})

char_key_to_value_map = {
    "All People": "DROP",
    "Education Attainment": "Education",
    "Work Experience": "WorkExperience",
    "Disability Status": "DisabilityStatus",
    "Residence": "Residence",
    "Age": "AgeBand",
    "Sex": "Gender",
    "Family Status": "FamilyStatus",
    "Race and Hispanic Origin": "Race",
    "Naitvity": "Nativity",
    "Region": "Region"
}

for key, df in pbsc_dfs.items():
    if key == "All People":
        continue;
    clean_column_names = {df.columns[0]:char_key_to_value_map[key], 
                          df.columns[1]: "2014 - Total", 
                          df.columns[2]: "2014 - Below Poverty", 
                          df.columns[3]: "2014 - Below Poverty MOE", 
                          df.columns[4]: "2014 - Below Poverty %", 
                          df.columns[5]: "2014 - Below Poverty % MOE", 
                          df.columns[6]: "2015 - Total", 
                          df.columns[7]: "2015 - Below Poverty", 
                          df.columns[8]: "2015 - Below Poverty MOE", 
                          df.columns[9]: "2015 - Below Poverty %", 
                          df.columns[10]: "2015 - Below Poverty % MOE", 
                          df.columns[11]: "2014-15 - Below Poverty Change", 
                          df.columns[12]: "2014-15 - Below Poverty % Change"
                         }
    clean_df = df.rename(columns=clean_column_names)[[char_key_to_value_map[key],
                                                      "2014 - Total", "2014 - Below Poverty",
                                                      "2015 - Total", "2015 - Below Poverty"]]
    
    y2014_clean_df = clean_df[[char_key_to_value_map[key],"2014 - Total", "2014 - Below Poverty"]]
    y2014_normalized_columns = {"2014 - Total": "Total", "2014 - Below Poverty":"BelowPoverty"}
    y2014_clean_df = y2014_clean_df.rename(columns=y2014_normalized_columns)
    y2015_clean_df = clean_df[[char_key_to_value_map[key],"2015 - Total", "2015 - Below Poverty"]]
    y2015_normalized_columns = {"2015 - Total": "Total", "2015 - Below Poverty":"BelowPoverty"}
    y2015_clean_df = y2015_clean_df.rename(columns=y2015_normalized_columns)
    
    clean_df = pd.concat([y2014_clean_df, y2015_clean_df])
    
    for col in all_pbsc_df.columns:
        if col not in clean_df.columns:
            clean_df[col] = "" ## Add empty columns to transform to common structure
    
    all_pbsc_df = pd.concat([all_pbsc_df, clean_df], sort=False)
    
print(all_pbsc_df.count())
all_pbsc_df.head()

Year                86
Race                86
Education           86
WorkExperience      86
DisabilityStatus    86
Residence           86
AgeBand             86
Nativity            86
Gender              86
FamilyStatus        86
Region              86
Total               86
BelowPoverty        86
dtype: int64


,Year,Race,Education,WorkExperience,DisabilityStatus,Residence,AgeBand,Nativity,Gender,FamilyStatus,Region,Total,BelowPoverty
0,,,"Total, aged 25 and older",,,,,,,,,212132.0,25163.0
1,,,No high school diploma,,,,,,,,,24582.0,7098.0
2,,,"High school, no college",,,,,,,,,62575.0,8898.0
3,,,"Some college, no degree",,,,,,,,,56031.0,5719.0
4,,,Bachelor's degree or higher,,,,,,,,,68945.0,3449.0


# Combine All Data
Here we combine DFs extracted from both excel workbooks into one consolidated DF.

In [11]:
for col in all_pbsc_df.columns:
    if col not in all_pbr_df.columns:
        all_pbr_df[col] = ""
        

all_data_df = pd.concat([all_pbr_df, all_pbsc_df], sort=False)
print(all_data_df.count())
all_data_df.head()

Year                1019
AgeBand             1019
Race                1019
Total               1019
BelowPoverty        1019
Education           1019
WorkExperience      1019
DisabilityStatus    1019
Residence           1019
Nativity            1019
Gender              1019
FamilyStatus        1019
Region              1019
dtype: int64


,Year,AgeBand,Race,Total,BelowPoverty,Education,WorkExperience,DisabilityStatus,Residence,Nativity,Gender,FamilyStatus,Region
0,1959,Under age 18,all races,64315,17552,,,,,,,,
1,1960,Under age 18,all races,65601,17634,,,,,,,,
2,1961,Under age 18,all races,66121,16909,,,,,,,,
3,1962,Under age 18,all races,67722,16963,,,,,,,,
4,1963,Under age 18,all races,69181,16005,,,,,,,,


# Prepare Dimensions
This is a generic reusable function which is used to extract unique dimension values and create a dataframe with columns that match DB columns.

In [12]:
def prepare_dim_df(dim):
    """
       This function returns a dimesion dataframe with unique dimension records
    """
    dim_info = []
    for index, value in enumerate(all_data_df[dim].unique()):
        if value is None or (type(value) == str and len(value) == 0):
            continue
        dim_info_row = OrderedDict()
        dim_info_row['ID'] =  index
        dim_info_row['CODE'] =  str(value)
        dim_info_row['DESCRIPTION'] = value
        dim_info.append(dim_info_row)
    
    dim_df = pd.DataFrame(dim_info)

    return dim_df

# Pre pare Disability Status

In [13]:
disabilitystatus_df = prepare_dim_df('DisabilityStatus')
disabilitystatus_df.head()

,ID,CODE,DESCRIPTION
0,1,"Total, aged 18 to 64","Total, aged 18 to 64"
1,2,With a disability,With a disability
2,3,With no disability,With no disability


# Prepare Family Status

In [14]:
familystatus_df = prepare_dim_df('FamilyStatus')
familystatus_df.head()

,ID,CODE,DESCRIPTION
0,1,In families,In families
1,2,Householder,Householder
2,3,Related children under age 18,Related children under age 18
3,4,Related children under age 6,Related children under age 6
4,5,In unrelated subfamilies,In unrelated subfamilies


# Create Population DF

Here we update the combined dataframe with dimension IDs.

In [15]:
def update_value_with_id(main_df, dim, dim_df):
    """
    This function replaces dimension values with corresponding IDs
    """
    di = dim_df.set_index('DESCRIPTION').to_dict()['ID']
    main_df[dim] = pd.Series(main_df[dim].map(di), dtype=object)
    return main_df

In [16]:
population_df = all_data_df.copy()
population_df = update_value_with_id(population_df, "AgeBand", ageband_df)
population_df = update_value_with_id(population_df, "DisabilityStatus", disabilitystatus_df)
population_df = update_value_with_id(population_df, "Education", education_df)
population_df = update_value_with_id(population_df, "FamilyStatus", familystatus_df)
population_df = update_value_with_id(population_df, "Gender", gender_df)
population_df = update_value_with_id(population_df, "Nativity", nativity_df)
population_df = update_value_with_id(population_df, "Race", race_df)
population_df = update_value_with_id(population_df, "Region", region_df)
population_df = update_value_with_id(population_df, "Residence", residence_df)
population_df = update_value_with_id(population_df, "WorkExperience", workexperience_df)
population_df = update_value_with_id(population_df, "Year", year_df)
population_df.head()

NameError: name 'ageband_df' is not defined

# Connect to DB
Here we connect to the database and explore table names.

In [ ]:
USERNAME="root"
PASSWORD="Mamjakin2012?"
HOST="localhost"
PORT="3306"
SCHEMA="CENTRANZ"
connection_string = f"{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{SCHEMA}"
engine = create_engine(f'mysql://{connection_string}')
engine.table_names()

# Insert Dimensions
Here we insert all dimensions into DB.

In [ ]:
disabilitystatus_df.set_index('ID').to_sql(name='DISABILITY_STATUS', con=engine, if_exists='append')

In [ ]:
familystatus_df.set_index('ID').to_sql(name='FAMILY_STATUS', con=engine, if_exists='append')

# Fix Population DF column names to match DB column names
Here we do handle of important things.

Change all the column names to match DB column names

By default, Pandas writes columns with integer values with some null values as floats. In our case, we have FK columns which are nullable. That is some of the rows have non null integers and some rows have null values. So default behaviour coverts all FK integer values into floats (i.e. 0 becomes 0.0, 1 becomes 1.0, etc). This behavior can be overriden by passing dtype to to_sql call.

In below code segment we use SQLAlchemy inspect API to get construct dtype dictionary.

In [ ]:
population_df.columns

In [ ]:
inspector = inspect(engine)
population_columns = inspector.get_columns('POPULATION')
dtypes = {}
for col in population_columns:
    if col['name'] == 'ID':
        continue
    dtypes[col['name']] = col['type']
dtypes

In [ ]:
column_name_map = {}
column_name_map['Total'] = "TOTAL"
column_name_map['BelowPoverty'] = "BELOW_POVERTY"
column_name_map['AgeBand'] = "AGE_BAND_FK"
column_name_map['DisabilityStatus'] = "DISABILITY_STATUS_FK"
column_name_map['Education'] = "EDUCATION_FK"
column_name_map['FamilyStatus'] = "FAMILY_STATUS_FK"
column_name_map['Gender'] = "GENDER_FK"
column_name_map['Nativity'] = "NATIVITY_FK"
column_name_map['Race'] = "RACE_FK"
column_name_map['Region'] = "REGION_FK"
column_name_map['Residence'] = "RESIDENCE_FK"
column_name_map['WorkExperience'] = "WORK_EXPERIENCE_FK"
column_name_map['Year'] = "YEAR_FK"

In [ ]:
final_population_df = population_df.rename(columns=column_name_map)

In [ ]:
final_population_df.dtypes

# Drop rows with (NA) in TOTAL or BELOW_POVERTY
There are some rows which have a string value (NA), where data is not available. We drop such rows.

In [ ]:
final_population_df = final_population_df[final_population_df['TOTAL']!='(NA)']
final_population_df = final_population_df[final_population_df['BELOW_POVERTY']!='(NA)']

# Insert Population DF

In [ ]:
final_population_df.to_sql(name='POPULATION', con=engine, if_exists='append', index=False, dtype=dtypes)